In [ ]:
!pip install --upgrade --quiet langchain-community langchain-openai neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["NEO4J_URI"] = ""
os.environ["NEO4J_USERNAME"] = ""
os.environ["NEO4J_PASSWORD"] = ""

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/sample_data/movies_small.csv')
df.head()

,movieId,title,actors,director,genres,imdbRating
0,1,Toy Story,Jim Varney|Tim Allen|Tom Hanks|Don Rickles,John Lasseter,Adventure|Animation|Children|Comedy|Fantasy,8.3
1,2,Jumanji,Robin Williams|Bradley Pierce|Kirsten Dunst|Jo...,Joe Johnston,Adventure|Children|Fantasy,6.9
2,3,Grumpier Old Men,Walter Matthau|Ann-Margret|Jack Lemmon|Sophia ...,Howard Deutch,Comedy|Romance,6.6
3,4,Waiting to Exhale,Whitney Houston|Lela Rochon|Angela Bassett|Lor...,Forest Whitaker,Romance|Drama|Comedy,5.6
4,5,Father of the Bride Part II,Steve Martin|Kimberly Williams-Paisley|Diane K...,Charles Shyer,Comedy,5.9


In [ ]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

# # Import movie information

# movies_query = """
# LOAD CSV WITH HEADERS FROM
# 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
# AS row
# MERGE (m:Movie {id:row.movieId})
# SET m.released = date(row.released),
#     m.title = row.title,
#     m.imdbRating = toFloat(row.imdbRating)
# FOREACH (director in split(row.director, '|') |
#     MERGE (p:Person {name:trim(director)})
#     MERGE (p)-[:DIRECTED]->(m))
# FOREACH (actor in split(row.actors, '|') |
#     MERGE (p:Person {name:trim(actor)})
#     MERGE (p)-[:ACTED_IN]->(m))
# FOREACH (genre in split(row.genres, '|') |
#     MERGE (g:Genre {name:trim(genre)})
#     MERGE (m)-[:IN_GENRE]->(g))
# """

# graph.query(movies_query)

In [ ]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Colour {name: STRING}
Relationship properties:

The relationships:
(:Colour)-[:sibling]->(:Colour)
(:Colour)-[:grandparent]->(:Colour)
(:Colour)-[:parent]->(:Colour)
(:Colour)-[:same_as]->(:Colour)
(:Colour)-[:siblings]->(:Colour)
(:Colour)-[:unknown]->(:Colour)


In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "what is the label of the relationship between black and white. dont forget to return"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c1:Colour {name: "black"})-[r]->(c2:Colour {name: "white"})
RETURN TYPE(r) AS relationship_type

Full Context:
[{'relationship_type': 'same_as'}]

> Finished chain.


In [ ]:
response['result']

'The label of the relationship between black and white is "same_as".'